# Tutorial
Put together by Kerem Okyay\
Sources: 
https://ccrma.stanford.edu/~njb/teaching/sstutorial/part2.pdf\
https://musicinformationretrieval.com



**decompose a matrix as a product of two or more matrices** 

                                    V = HW
- A is the data, audio spectrogram.(FxT)
- W is the basis vectors or dictionary. A single column is referred to as a basis vector. (FxK)
- H is the weights or the activation matrix. A row of H represents the gain of the corresponding basis vector. (KxT)


Columns of V as weighted sum of basis vectors.

In [1]:
%matplotlib inline
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
import scipy
import soundfile as sf

In [2]:
x, fs = librosa.load("../data/grandpiano.wav",duration=6)
ipd.Audio(x,rate=fs)

/Users/keremokyay/miniforge3/envs/spai/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: '../data/grandpiano.wav'

In [ ]:
plt.figure(figsize=(14,5))
librosa.display.waveshow(x,sr=fs)

In [ ]:
print(x.shape)
print(fs)

### Short-Time Fourier Transform (STFT)
inputs time domain signal x\
outputs Magnitude |X| and phase <X matrices

The **short-time Fourier transform (STFT)** ([Wikipedia](https://en.wikipedia.org/wiki/Short-time_Fourier_transform); FMP, p. 53) is obtained by computing the Fourier transform for successive frames in a signal. 

$$ X(m, \omega) = \sum_n x(n) w(n-m) e^{-j \omega n} $$

As we increase $m$, we slide the window function $w$ to the right. For the resulting frame, $x(n) w(n-m)$, we compute the Fourier transform. Therefore, the STFT $X$ is a function of both time, $m$, and frequency, $\omega$.

In [ ]:
X = librosa.stft(x)
# defaults: 
# n_fft(length of windowed signal after padding with zeros):2048
# hop_length(number of audio samples between adjacent STFT colums): win_length // 4
# win_length(each frame of audio is windowed by window of length win_length
# and them padded with zeros to match n_fft): n_fft = 2048
# window: scipy.signal.windows.hann

### Spectrogram
In music processing, we often only care about the spectral magnitude and not the phase content.

The **spectrogram** ([Wikipedia](https://en.wikipedia.org/wiki/Spectrogram); FMP, p. 29, 55) shows the the intensity of frequencies over time. A spectrogram is simply the squared magnitude of the STFT:

$$ S(m, \omega) = \left| X(m, \omega) \right|^2 $$

In [ ]:
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14,5))
librosa.display.specshow(Xdb, sr=fs, x_axis='time', y_axis='hz')

 ### NMF
 inputs |X|, outputs W and H

[Nonnegative matrix factorization](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization) (NMF) is an algorithm that factorizes a nonnegative matrix, $X$, into a product of two nonnegative matrices, $W$ and $H$. It is an unsupervised iterative algorithm that minimizes a distance between $X$ and the product $WH$:

$$ \min_{W, H} d(X, WH) $$

If $X$ has dimensions $M$ by $N$, then $W$ will have dimensions $M$ by $R$, and $H$ will have dimensions $R$ by $N$, where inner dimension $R$ is the *rank* or *number of components* of the decomposition.

When applied to a musical signal, we find that NMF can decompose the signal into separate note events. Therefore, NMF is quite useful and popular for tasks such as transcription and source separation.

The input, $X$, is often a magnitude spectrogram. In such a case, we find that the columns of $W$ represent spectra of note events, and the rows of $H$ represent temporal envelopes of the same note events.

In [ ]:
X_mag, X_phase = librosa.magphase(X)
n_components = 10 # change this to better understand what happens
W, H = librosa.decompose.decompose(X_mag,n_components=n_components, sort=True)

## Notice how similar the following plot to a piano roll represantation.
## These are the weights plotted against time.

However i do not understand fully what is the unit of the y axis


In [ ]:
plt.figure(figsize=(14,5))
librosa.display.specshow(librosa.amplitude_to_db(np.dot(W,H)), sr=fs, x_axis='time', y_axis='hz')
# hz is the midi pitch 22 45 45 8 62 62
H.shape

In [ ]:
# these are the basis vectors plotted against time 
Wdb = librosa.amplitude_to_db(W)
plt.figure(figsize=(14,5))
librosa.display.specshow(Wdb, sr=fs, x_axis='time', y_axis='hz')

In [ ]:
print(X.shape)
print(W.shape)
print(H.shape)

Let's display the spectral profiles(basis vectors), $\{w_1, ..., w_R\}$:


In [ ]:
plt.figure(figsize=(13, 7))
for n in range(n_components):
    plt.subplot(5, 2, n+1)
    plt.plot(W[:,n])
    plt.ylim(-3, W.max())
    plt.xlim(0, W.shape[0])
    plt.ylabel('Component %d' % n)
    print(np.argmax(W[:,n]) * 11025/1025) # fs = 22050, only up to fs/2 can be detected properly

In [ ]:
print(W.max())

Let's display the temporal activations(gains corresponding with basis vectors), $\{h_1, ..., h_R\}$:

In [ ]:
plt.figure(figsize=(13, 7))
for n in range(n_components):
    plt.subplot(5, 2, n+1)
    plt.plot(H[n])
    plt.ylim(0, H.max())
    plt.xlim(0, H.shape[1])
    plt.ylabel('Component %d' % n)

### FILTER and Inverse Short-Time Fourier Transform (ISTFT)
FILTER\
choose a subset of basis vectors Ws and activations Hs to reconstruct source s\
ISTFT\
inputs |X| and phase <X matrices\
outputs time domain signal x

In [ ]:
Y = np.dot(W[:0], H[0:])
print(W.shape)
print(W[:,0].shape)
print(H[0].shape)
print(X_phase.shape)

In [ ]:
def generate_sine(f0, sr, n_duration):
    n = np.arange(n_duration)
    return 0.2*np.sin(2*np.pi*f0*n/float(sr))

In [ ]:
def estimate_pitch_and_generate_sine(x, onset_samples,f0, i, sr):
    n0 = onset_samples[i]
    n1 = onset_samples[i+1]
    return generate_sine(f0, sr, n1-n0)

In [ ]:
for n in range(n_components):
    
    # Re-create the STFT of a single NMF component. row of W with the column of H
    Y = np.outer(W[:,n],H[n]) 
    plt.figure(figsize=(14,5))
    #librosa.display.specshow(Y, sr=fs, x_axis='time', y_axis='hz')

    # Transform the STFT into the time domain.
    y = librosa.istft(Y* X_phase)
    # APPLY fast NLS estimator on this y

    f0 = np.argmax(W[:,n]) * 11025/1025 # fs = 22050, only up to fs/2 can be detected properly

    
    print('Component {}:'.format(n))
    #ipd.display( ipd.Audio(y, rate=fs) )

In [ ]:
Y = np.dot(W, H)*X_phase

# Transform the STFT into the time domain.
reconstructed_signal = librosa.istft(Y, length=len(x))
ipd.Audio(reconstructed_signal, rate=fs)
#librosa.display.waveshow(reconstructed_signal,sr=fs,)
#error
librosa.display.waveshow(x-reconstructed_signal,sr=fs)

In [ ]:
librosa.display.waveshow(x,sr=fs)
librosa.display.waveshow(reconstructed_signal,sr=fs)

In [ ]:
residual = x - reconstructed_signal
residual[0] = 1 # hack to prevent automatic gain scaling
#ipd.Audio(residual, rate=fs)
ipd.Audio(x, rate=fs)
